| Aplicativo | Play Store ID           | App Store ID |
| ---------- | ----------------------- | ------------ | 
| Claro      | com.nvt.cs              | 526829675    |
| TIM        | br.com.timbrasil.meutim | 668591218    |
| Vivo       | br.com.vivo             | 475915064    |



In [2]:
import requests
import pandas as pd
from google_play_scraper import Sort, reviews

Tem como pegar versão do app?

**Dados Play Store**

In [3]:
qtd_comentarios = 1000

dados_android = []

apps = {
    "Claro": "com.nvt.cs",
    "TIM": "br.com.timbrasil.meutim",
    "Vivo": "br.com.vivo"
}

for app, app_id in apps.items():
    result, _ = reviews(
        app_id,
        lang='pt',
        country='br',
        sort=Sort.NEWEST,
        count=qtd_comentarios
    )
        
    for review in result:
        dados_android.append({
            "app": app,
            "system": "Android",
            "user": review.get("userName", None),
            "score": review.get("score", None),
            "content": review.get("content", None)
        })

comentarios_android = pd.DataFrame(dados_android, columns=["app", "system", "user", "score", "content"])

comentarios_android.head()

,app,system,user,score,content
0,Claro,Android,deivissom peixoto,2,ótimo
1,Claro,Android,Deyvid Boyka,1,qual dificuldade do app indentificar chip e en...
2,Claro,Android,Danilo e Ana Esther Gomes,5,me ajuda de mais
3,Claro,Android,Andre luiz Oliveira monteiro,5,muito bom o app instalado .
4,Claro,Android,Gilvan Sousa,4,tá atendendo bem mim demanda


**Dados App Store**

In [7]:
page = 10

dados_ios = []

apps = {
    "Claro": "526829675",
    "TIM": "668591218",
    "Vivo": "475915064"
}

for app, id in apps.items():

    url = f'https://itunes.apple.com/rss/customerreviews/page={page}/id={id}/sortby=mostrecent/json'
    response = requests.get(url)
    data = response.json()

    if "feed" in data and "entry" in data["feed"]:
        for entry in data["feed"]["entry"][1:]: #1 é informações do app
            dados_ios.append({
                "app": app,
                "system": "iOS",
                "user": entry['author']['name']['label'],
                "score": entry['im:rating']['label'],
                "content": entry['content']['label']
            })

comentarios_ios = pd.DataFrame(dados_ios, columns=["app", "system", "user", "score", "content"])

comentarios_ios.head()

,app,system,user,score,content
0,Vivo,iOS,Gustavo Txas,1,"Esse App, pelo menos pra mim, ainda não funcio..."
1,Vivo,iOS,ffalcao16,2,"A proposta do app é boa, mas a cada consulta q..."
2,Vivo,iOS,gabizago,1,"Mesmo com o app, segue nao funcionando a conta..."
3,Vivo,iOS,Riic,2,Muito útil o aplicativo se funcionasse !\nPrec...
4,Vivo,iOS,Allannvictor,5,Recomendo!


In [8]:
comentarios = pd.concat([comentarios_android, comentarios_ios], ignore_index=True)
comentarios_android.shape, comentarios_ios.shape, comentarios.shape

((3000, 5), (21, 5), (3021, 5))